In [1]:
import sys
# appending a path
sys.path.append('../')

import numpy as np
import torch
import jax
import pychop
from numpy import linalg
from time import time
# from pychop.chop import chop
# from pychop.quant import quant
from time import time
np.random.seed(0)

In [2]:
X_np = np.random.randn(5000, 5000) # Numpy array
X_th = torch.Tensor(X_np) # torch array
X_jx = jax.numpy.asarray(X_np)
print(X_np)

[[ 1.76405235  0.40015721  0.97873798 ...  0.92918181  0.22941801
   0.41440588]
 [ 0.30972382 -0.73745619 -1.53691988 ...  0.51687218 -0.03292069
   1.29811143]
 [-0.20211703 -0.833231    1.73360025 ...  0.75309415 -0.58103281
  -0.19837974]
 ...
 [ 1.07432182  1.188486    0.5092741  ...  0.07053449  0.59975911
  -2.41029925]
 [ 0.32432475 -0.02337844  1.62873399 ... -0.16088168 -1.59772992
   1.414703  ]
 [ 0.63460807  1.38090977  0.54829109 ...  0.30762729 -0.11078251
   0.83859307]]


### print unit-roundoff in machine

In [3]:
from pychop import float_params

float_params()

,,u,xmins,xmin,xmax,p,emins,emin,emax
0,q43,6.25e-02,1.95e-03,1.56e-02,2.40e+02,4,-9,-6,7
1,q52,1.25e-01,1.53e-05,6.10e-05,5.73e+04,3,-16,-14,15
2,b,3.91e-03,9.18e-41,1.18e-38,3.39e+38,8,-133,-126,127
3,h,4.88e-04,5.96e-08,6.10e-05,6.55e+04,11,-24,-14,15
4,t,4.88e-04,1.15e-41,1.18e-38,3.40e+38,11,-136,-126,127
5,s,5.96e-08,1.40e-45,1.18e-38,3.40e+38,24,-149,-126,127
6,d,1.11e-16,4.94e-324,2.23e-308,1.80e+308,53,-1074,-1022,1023
7,q,9.63e-35,0.00e+00,0.00e+00,inf,113,-16494,-16382,16383


### set backend

In [4]:
pychop.backend('torch')

In [5]:
pychop.backend('numpy', 1) # print information, NumPy is the default option.

Load NumPy backend.


### run chop

In [6]:
from pychop.chop import Chop
pyq_f = Chop('h')
pyq_f(X_np)

array([[ 1.76367188,  0.40014648,  0.97851562, ...,  0.92919922,
         0.22937012,  0.41430664],
       [ 0.30981445, -0.73730469, -1.53710938, ...,  0.51708984,
        -0.03292847,  1.29785156],
       [-0.20214844, -0.83300781,  1.73339844, ...,  0.75292969,
        -0.58105469, -0.19836426],
       ...,
       [ 1.07421875,  1.18847656,  0.50927734, ...,  0.07055664,
         0.59960938, -2.41015625],
       [ 0.32421875, -0.02337646,  1.62890625, ..., -0.16088867,
        -1.59765625,  1.41503906],
       [ 0.63476562,  1.38085938,  0.54833984, ...,  0.30761719,
        -0.11077881,  0.83837891]])

In [7]:
print("2^"+str(int(np.log2(pyq_f.u))))

2^-11


In [8]:
pychop.backend('torch', 1) # print information

pyq_f = Chop('h')
st = time()
pyq_f(X_th)
print("runtime:", time() - st)

Load Troch backend.
runtime: 0.9297142028808594


In [9]:
pychop.backend('torch')
pyq_f = Chop('h')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_gpu = X_th.to(device)


st = time()
pyq_f(X_gpu)
print("runtime:", time() - st)

runtime: 0.877464771270752


In [10]:
pychop.backend('jax', 1) # print information
pyq_f = Chop('h')
pyq_f(X_jx)

Load JAX backend.


Array([[ 1.7636719 ,  0.40014648,  0.9785156 , ...,  0.9291992 ,
         0.22937012,  0.41430664],
       [ 0.30981445, -0.7373047 , -1.5371094 , ...,  0.51708984,
        -0.03292847,  1.2978516 ],
       [-0.20214844, -0.8330078 ,  1.7333984 , ...,  0.7529297 ,
        -0.5810547 , -0.19836426],
       ...,
       [ 1.0742188 ,  1.1884766 ,  0.50927734, ...,  0.07055664,
         0.5996094 , -2.4101562 ],
       [ 0.32421875, -0.02337646,  1.6289062 , ..., -0.16088867,
        -1.5976562 ,  1.4150391 ],
       [ 0.6347656 ,  1.3808594 ,  0.54833984, ...,  0.3076172 ,
        -0.11077881,  0.8383789 ]], dtype=float32)

In [11]:
pychop.backend('numpy')
pyq_f = Chop('h', rmode=5, flip=0)
pyq_f(X_np)

array([[ 1.76367188,  0.40014648,  0.97851562, ...,  0.92919922,
         0.22937012,  0.41430664],
       [ 0.30981445, -0.73779297, -1.53710938, ...,  0.51660156,
        -0.03292847,  1.29785156],
       [-0.20202637, -0.83349609,  1.73339844, ...,  0.75341797,
        -0.58105469, -0.19836426],
       ...,
       [ 1.07421875,  1.18847656,  0.50927734, ...,  0.07049561,
         0.59960938, -2.41015625],
       [ 0.32421875, -0.02339172,  1.62890625, ..., -0.1607666 ,
        -1.59765625,  1.41503906],
       [ 0.63476562,  1.38085938,  0.54833984, ...,  0.30761719,
        -0.11077881,  0.83886719]])

## For customized precisions


In [12]:
from pychop import Customs
prec = Customs(t=2, emax=10) # use precision 2 and set maximum exponenet of 10
# Second, define parameter customs instead of prec,

x = np.random.rand(1000, 1000) # use x = torch.rand(size=(10000, 10000)) for Torch backend
nc = Chop(customs=prec, rmode=3)
y = nc(x)
print(y[0, :5])


## To print out the unit-roundoff information, simply set verbose=1, use

pyq_f = Chop('h', verbose=1)
## The result is:

## The floating point format is with unit-roundoff of 4.882812e-04 (≈2^-11).

[0.5  0.5  0.75 0.75 0.25]
The floating point format is with unit-roundoff of 4.882812e-04 (≈2^-11).
